<a href="https://colab.research.google.com/github/NicolasSpettel/lab-natural-language-to-sql/blob/main/lab_natural_language_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [3]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-j0oweawn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-j0oweawn
  Resolved https://github.com/huggingface/accelerate.git to commit 979d81e4a92d08861f34fa6fbb66480b20771e76
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.11.0.dev0-py3-none-any.whl size=375177 sha256=f4e1d66d7c28cec999f1c3cbc902008f3ce19fe367915e0cded6fc74483a61c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-cw4e_4cp/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [5]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [6]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [7]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [9]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5 # trying five different options (kind of like cross validaiton)
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [10]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);

    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);

    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [11]:
sp_nl2sql = sp_nl2sql.format(question="Show me the last five hires, order them by salary, having the highest paid at top")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);
             
    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);
             
    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Show me the last five hires, order them by salary, having the highest paid at top`:
    ```sql3
    


In [12]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [13]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [14]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, salaries.salary, salaries.effective_date FROM salaries JOIN employees ON salaries.employee_id = employees.employee_id ORDER BY salaries.salary DESC NULLS LAST LIMIT 5;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [15]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);

    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);

    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question.

    what employee has the highest salary?

    SELECT
    employee_id, first_name, last_name
    FROM
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 1;

    `{question}`:
    ```sql3
    """


In [16]:
sp_nl2sql2 = sp_nl2sql2.format(question="Show me the last five hires, order them by salary, having the highest paid at top")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);
             
    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);
             
    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question.

    wha

In [17]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [18]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT 
    first_name, last_name, hire_date, salary
    FROM 
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 5;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [19]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);

    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);

    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);

    ### Samples
    what employee has the highest salary?

    SELECT
    employee_id, first_name, last_name
    FROM
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 1;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [20]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Show me the last five hires, order them by salary, having the highest paid at top")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);
             
    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);
             
    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);
    
    ### Samples
    what employee has the highest salary?

    SELECT 
    employee_id, first_name, last_name
    FRO

In [21]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [22]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT 
    first_name, last_name, hire_date
    FROM 
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 5;


#Now the question in spanish.


In [23]:
sp_nl2sql3 = sp_nl2sql3b.format(question="¿Muéstrame las últimas cinco contrataciones, ordénalas por salario, teniendo el mejor pagado en la parte superior?")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    hire_date DATE,
    department_id INTEGER);
             
    CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT);
             
    CREATE TABLE salaries (
    employee_id INTEGER PRIMARY KEY,
    salary REAL,
    effective_date DATE);
    
    ### Samples
    what employee has the highest salary?

    SELECT 
    employee_id, first_name, last_name
    FRO

In [24]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [25]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT 
    e.employee_id, e.first_name, e.last_name
    FROM 
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.first_name, employees.last_name, salaries.salary, salaries.effective_date FROM salaries JOIN employees ON salaries.employee_id = employees.employee_id ORDER BY salaries.salary DESC NULLS LAST LIMIT 5;


* SELECT
    first_name, last_name, hire_date, salary
    FROM
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 5;

* SELECT
    first_name, last_name, hire_date
    FROM
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 5;

* SELECT
    e.employee_id, e.first_name, e.last_name
    FROM
    employees e
    JOIN
    salaries s
    ON
    e.employee_id == s.employee_id
    ORDER BY
    salary DESC
    LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It handeled the spanish quite good, except the Limit that should be 5

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [52]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=1 # CHANGE TO 1 HERE TO BE FAST!
    )
    return outputs

In [54]:
sp_nl2sql4 = sp_nl2sql3b.format(question="How can I use SQL to find the average salary of employees hired after January 1, 2023, for each department?")

In [55]:
input_sentences = tokenizer(sp_nl2sql4, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [56]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT d.department_name, AVG(s.salary) AS average_salary
     FROM employees e
     JOIN departments d ON e.department_id = d.department_id
     JOIN salaries s ON e.employee_id = s.employee_id
     WHERE s.effective_date > '2023-01-01'
     GROUP BY d.;


In [57]:
sp_nl2sql5 = sp_nl2sql3b.format(question="What is the SQL query to list all employees' first and last names, their department name, and their current salary, ordered by last name?")

In [58]:
input_sentences = tokenizer(sp_nl2sql5, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [59]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.first_name, e.last_name, d.department_name, s.salary
    FROM employees e
    JOIN departments d
    ON e.department_id = d.department_id
    JOIN salaries s
    ON e.employee_id = s.employee_id
    ORDER BY e.last_name DESC;


In [60]:
sp_nl2sql6 = sp_nl2sql3b.format(question="If a new employee is hired, what information would I need to insert a new record into all three tables, and what would the SQL commands look like?")

In [61]:
input_sentences = tokenizer(sp_nl2sql6, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [62]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

;


Changing the max_new_tokesn to 100 in earlier quesiton turned out to be a fatal flaw, because the SQL query didn't actually finish for the first example. Number 2 went great i think and number three was asking for a INSERT command essentially and the model didn't give an answer for that.

Trying out the updated model

In [67]:
#The model weights were updated at 7 AM UTC on Feb 7, 2024. The new model weights lead to a much more performant model – particularly for joins.

#If you downloaded the model before that, please redownload the weights for best performance.

#Model Card for SQLCoder-7B-2
#A capable large language model for natural language to SQL generation.
model_name = "defog/sqlcoder-7b-2"
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [68]:
sp_nl2sql6 = sp_nl2sql3b.format(question="If a new employee is hired, what information would I need to insert a new record into all three tables, and what would the SQL commands look like?")

In [69]:
input_sentences = tokenizer(sp_nl2sql6, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [70]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

### thTER
    Based on your instructions, here is the SQL query I have generated to явля the question
    `If a new employee is hired;


In [71]:
sp_nl2sql6 = sp_nl2sql3b.format(question="Update the table for a new employee, name is john , last name doe, hire date today, department of your choice and id should be automatically")

In [72]:
input_sentences = tokenizer(sp_nl2sql6, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [73]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

### thTER
    Based on your instructions, here is the SQL query I have generated to явля the question
    `Update the table for a new;


In [74]:
sp_nl2sql6 = sp_nl2sql3b.format(question="find the oldest employee (date of hiring)")

In [75]:
input_sentences = tokenizer(sp_nl2sql6, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=100)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [76]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

### thTER
    Based on your;


This model does not perform at all, either im doing something wrong here.. but using russian letters seems strange